## The \_\_new__ method

In [2]:
class Point(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
      

p = object.__new__(Point)
p.__init__(10, 20)
p.__dict__

{'x': 10, 'y': 20}

In [4]:
class Point:
    def __new__(cls, x, y):
        print('Creating instance...', x, y)
        return super().__new__(cls)
    
    def __init__(self, x, y) -> None:
        print('Init called...', x, y)
        self.x = x
        self.y = y
        
p = Point(10, 20)

Creating instance... 10 20
Init called... 10 20


In [7]:
class Person:
    def __new__(cls, name):
        print(f'Person: instantiating {cls.__name__}...')
        return object.__new__(cls)
    
    def __init__(self, name):
        print('Person: initializing instance...')
        self.name = name
        
class Student(Person):
    def __new__(cls, name, major):
        print(f'Student: instantiating {cls.__name__}...')
        return super().__new__(cls, name)
    
    def __init__(self, name, major):
        print('Student: initializing instance')
        super().__init__(name)
        self.major = major


s = Student('John', 'Math')
s.__dict__

Student: instantiating Student...
Person: instantiating Student...
Student: initializing instance
Person: initializing instance...


{'name': 'John', 'major': 'Math'}

In [8]:
class Square:
    # new is a static method. we don't really need the @staticmethod 
    # decorator because Python will do that automaticaly
    @staticmethod
    def __new__(cls, w, l):
        cls.area = lambda self: self.w * self.l
        instance = super().__new__(cls)
        instance.w = w
        instance.l = l
        return instance
    
s = Square(2, 3)
print(s.__dict__)
s.area()

{'w': 2, 'l': 3}


6

## How classes are created
- When Python encounters a **`class`** (Person) as it compiles (executes) our code:
    * a symbol **`Person`** is created in the namespace
    * that symbol is a reference to the class **`Person`** (it is an object)
- How does Python create that class?
    * a class is an instance of **`type`**:
        - that's why a **`class`** is also called a **`type`**  
        - type is a **`callable`** (and is in fact a type (class) itself, and inherits from **`object`**)

In [15]:
import math

class_name = 'Circle'

class_body = """
def __init__(self, x, y, r):
    self.x = x
    self.y = y
    self.r = r
    
def area(self):
    return math.pi * self.r **2
"""

class_bases = ()
class_dict = {}

# populate the dict with the content inside class body
exec(class_body, globals(), class_dict)

# Create the class
# if type gets called with only one argument, it returns the type of the object
# if we supply three arguments, then the type function returns a new type instance
Circle = type(class_name, class_bases, class_dict)

c = Circle(x=0, y=0, r=1)
c.area()

3.141592653589793

## Inheriting from type

- type is an **`object`**
    * like any object it inherits from **`object`**
    * it has \_\_new__ and \_\_init__
<br><br>

- type is a **`class`**
    * it is callable
    * calling it creates a new instance of type
    * just like calling any class
        - calls \_\_new__
    * can be used as a **`base class`** for a custom class
        - we can overide \_\_new__ --> tweak things, but delegate to type for actual type creation

In [17]:
import math

# Meta class
class CustomType(type):
    def __new__(cls, name, bases, class_dict):
        print('Customized type creation')
        cls_obj = super().__new__(cls, name, bases, class_dict)
        cls_obj.circ = lambda self: 2 * math.pi * self.r
        return cls_obj    
  
    
class_name = 'Circle'

class_body = """
def __init__(self, x, y, r):
    self.x = x
    self.y = y
    self.r = r
    
def area(self):
    return math.pi * self.r **2
"""

class_bases = ()
class_dict = {}

# populate the dict with the content inside class body
exec(class_body, globals(), class_dict)

Circle = CustomType('Circle', class_bases, class_dict)

c = Circle(x=0, y=0, r=1)
print(c.circ())
print(c.area())

Customized type creation
6.283185307179586
3.141592653589793


### An alterntive way of writing the CustomType class

In [ ]:
class CustomType(type):
    def __new__(cls, name, bases, class_dict):
        print('Customized type creation')
        class_dict['circ'] = lambda self: 2 * math.pi * self.r
        return super().__new__(cls, name, bases, class_dict)  

## Metaclasses
- To create a class, another class is used (typically **`type`**)
- The class used to create a class, is called the **`metaclass`** of that class
    * by default, Python uses **`type`** as the metaclass
    * but we can override this --> class Person(**`metaclass=MyType`**)

In [22]:
import math

class CustomType(type):
    def __new__(mcls, name, bases, class_dict):
        print(f'Using custom metaclass "{mcls.__name__}" to create class "{name}"...')
        cls_obj = super().__new__(mcls, name, bases, class_dict)
        cls_obj.circ = lambda self: 2 * math.pi * self.r
        return cls_obj  
    
class Circle(metaclass=CustomType):
    def __init__(self, x, y, r):
        self.x = x
        self.y = y
        self.r = r
    
    def area(self):
        return math.pi * self.r **2
    


Using custom metaclass "CustomType" to create class "Circle"...
